# Create Nested Dictionary (JSON) Dataset:

**`with Mr Fugu Data Science`**


# (◕‿◕✿)

[Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw/) | [Github](https://github.com/MrFuguDataScience)


# Objective and Outcome:

+ Create a Nested Dictionary (Nested JSON) format
    + Understand pitfalls when using Dataframes to create nested JSON objects
        + Evaluate Dataframes stored as json/dict .csv problems
+ Create a Function to store our data properly
    + Describe the format and ability to finally send to Mongo if you needed.

In [1]:
import faker        # Crete fake People
import pandas as pd # Dataframes
import random       # random number generation
import requests     # grab web-page
from bs4 import BeautifulSoup as bsopa  #parse web-page 
import re # regex parsing

In [2]:
"""
I want to generate skills of employees ranging from 1:(1-len(skills) list).
I am generating 500 entries, because I will populate 500 fake prospective employees
to use in my Fake Employment Recruiter Database today.

"""

skills=['Python','Java','SQL','MongoDB','R','C++','Spark','TensorFlow','skLearn']


def create_rndm_sklls(skills):
    random.seed(300) # USE TO KEEP DATA CONSISTENT
    randomlist = []
    for i in range(0,500):
        n = random.randint(1,len(skills)) # mention this!
        randomlist.append(n)
#     print(randomlist)

    lst_skills_per_person=[]
    for i in randomlist:
        lst_skills_per_person.extend([random.sample(skills, i)])
    lst_skills_per_person
    return lst_skills_per_person

create_rndm_sklls(skills)[:5]

[['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
 ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
 ['Spark'],
 ['skLearn', 'SQL', 'R', 'Spark', 'Java', 'C++', 'Python', 'TensorFlow'],
 ['TensorFlow', 'C++', 'SQL', 'Java', 'R', 'MongoDB']]

In [3]:
fake_=faker.Faker()
fake_.seed(413)
fake_name=[]
for i in range(500):
    if i not in fake_name:
        fake_name.append(fake_.name().split(' '))
        
fake_name[:5]

[['Margaret', 'Mcdonald'],
 ['Michael', 'Carter'],
 ['Brenda', 'Tyler'],
 ['Joseph', 'King'],
 ['Laura', 'Webb']]

In [4]:
response=requests.get("https://www.softschools.com/social_studies/state_abbreviations/")

response.status_code # checking that the request was (ok=200)

soupie=bsopa(response.text,'lxml')  # converting format

html_source=soupie.find('div',{"class":"middlecol"}) # html source code

# tags=html_source.find('a').find_all('href')


In [5]:
# Get US State Abreviations:

table = html_source.find('table' ,attrs={'class':'colorBgGreen'})
first_td = table.find('tr')
text=first_td.renderContents()
text_=text.decode("utf-8") # convert bytes to string
jj=text_.split('\n')

state_td_tags=[]
for i in jj:
    if re.findall(r'\b[A-Z]{2}\b',i):
#         trr.append(i.replace('<td>','').replace('</td>',''))
        state_td_tags.append(i.replace('<td>','').replace('</td>',''))
    
state_abbr=state_td_tags[:-2] #the last two lines i didn't need US Territories
# state_abbr


In [6]:
state_rnd=random.choices(state_abbr,k=500)
# state_rnd

In [7]:
# Relocation: (Employee willing to relocate?)
willing_to_relocate=['yes','no','maybe']
relocation_=random.choices(willing_to_relocate,k=500)

In [8]:
# Nested Data:

specialty=['Machine Learning','Data Visualization','Database','Statistics']
experience_level=['Junior','Mid','Senior']

spec_=random.choices(specialty,k=500)
exper_=random.choices(experience_level,k=500)
sp=['specialty']*500
exp=['experience']*500
my_list=list(zip(sp,spec_))
my_list2=list(zip(exp,exper_))


dict_specialty=[{key: value} for (key, value) in my_list]

dict_exper=[{key: value} for (key, value) in my_list2]

# This will be our tuple of dictionary pairs:
spec_exp=list(zip(dict_specialty,dict_exper))
# my_list
# dict_specialty
# spec_exp

In [9]:
# Generating, the full dataset

first_name=[]
last_name=[]
for i in fake_name:
    first_name.append(i[0])
    last_name.append(i[1])


nested_ppl=list(zip(first_name,last_name,create_rndm_sklls(skills),
                    state_rnd,spec_exp,relocation_))
nested_ppl[:2]

[('Margaret',
  'Mcdonald',
  ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
  'AL',
  ({'specialty': 'Database'}, {'experience': 'Mid'}),
  'no'),
 ('Michael',
  'Carter',
  ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
  'AR',
  ({'specialty': 'Statistics'}, {'experience': 'Senior'}),
  'yes')]

In [10]:
cols=['first_name','last_name','skills','state','specialty_exper','relocation']
job_seekers_=pd.DataFrame(nested_ppl,columns=cols,)

pd.DataFrame.to_csv(job_seekers_,'job_seekers.csv')
job_seekers_.head()

,first_name,last_name,skills,state,specialty_exper,relocation
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,"({'specialty': 'Database'}, {'experience': 'Mi...",no
1,Michael,Carter,"[TensorFlow, R, Spark, MongoDB, C++, SQL]",AR,"({'specialty': 'Statistics'}, {'experience': '...",yes
2,Brenda,Tyler,[Spark],UT,"({'specialty': 'Database'}, {'experience': 'Mi...",no
3,Joseph,King,"[skLearn, SQL, R, Spark, Java, C++, Python, Te...",FL,"({'specialty': 'Machine Learning'}, {'experien...",maybe
4,Laura,Webb,"[TensorFlow, C++, SQL, Java, R, MongoDB]",WY,"({'specialty': 'Machine Learning'}, {'experien...",maybe


# Creating Actual `Nested Dictionary (JSON)` format for MongoDB

In [19]:
candidates_=list(zip(first_name,last_name,create_rndm_sklls(skills),
                    state_rnd,spec_,exper_,relocation_))
cols_=['first_name','last_name','skills','state','specialty','experience','relocation']

un_nested_candidates=pd.DataFrame(candidates_,columns=cols_)
un_nested_candidates.head()

,first_name,last_name,skills,state,specialty,experience,relocation
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,Database,Mid,no
1,Michael,Carter,"[TensorFlow, R, Spark, MongoDB, C++, SQL]",AR,Statistics,Senior,yes
2,Brenda,Tyler,[Spark],UT,Database,Mid,no
3,Joseph,King,"[skLearn, SQL, R, Spark, Java, C++, Python, Te...",FL,Machine Learning,Senior,maybe
4,Laura,Webb,"[TensorFlow, C++, SQL, Java, R, MongoDB]",WY,Machine Learning,Junior,maybe


In [14]:
def df_to_nested_json(df, candidate, lat='state'):
    _json = {'features':[]}
    
    for _, row in df.iterrows():
        feature = {'candidate':{}}
#                    'specialty_experience':{}
#         feature['specialty_experience']['state'] = [row[lat]]
#         feature['specialty_experience']['skills'] = [row[lon]
        for prop in candidate:
            feature['candidate'][prop] = row[prop]
        _json['features'].append(feature)
    return _json

#-------------------------

cols=un_nested_candidates.columns

people_json=df_to_nested_json(un_nested_candidates, cols)

people_json['features'][:2]

people_json

dict

In [15]:
import json

with open('employee_data.json', 'w') as fp:
    json.dump(people_json, fp)